<a href="https://colab.research.google.com/github/aravindskumar98/DL/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [12]:
## device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
##hyper parameters
num_epochs = 4
batch_size = 4 
learning_rate = 0.001

# dataset has PILImage images of range [0,1]
# we transform them to tensors of normalised range [-1,1]

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [14]:
train_dataset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform)
test_dataset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = batch_size, shuffle= True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size = batch_size, shuffle= False)

In [16]:
classes =('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

In [18]:
## implement conv net
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet,self).__init__()
    self.conv1 = nn.Conv2d(3,6,5) ##(color channels, output channel size, kernel size (5*5))
    self.pool = nn.MaxPool2d(2,2) ##(kernel size, stride)
    self.conv2 = nn.Conv2d(6,16,5)
    self.fc1 = nn.Linear(16*5*5, 120) ## final size of each channel is 5*5 <--------------IMPORTANT!
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def forward(self,x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1,16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
    
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)



In [20]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i,(images,labels) in enumerate(train_loader):
    ##origin shape: [4,3,32,32]
    ##input layer: 3 input channels, 6 output channels, 5 kernel size
    images = images.to(device)
    labels = labels.to(device)

    #forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    #backwar and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1)%2000 ==0:
      print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')

print('Finished Training')

with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_correct = [0 for _ in range(10)]
  n_class_samples = [0 for _ in range(10)]

  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    #max returns val, index
    _,predicted = torch.max(outputs,1)
    n_samples += labels.size(0)
    n_correct += (predicted==labels).sum().item()

    for i in range(batch_size):
      label = labels[i]
      pred = predicted[i]
      if label == pred:
        n_class_correct[label]+=1
      n_class_samples[label]+=1

  acc = 100 * n_correct / n_samples
  print("Accuracy of the network : ", acc,"%")

  for i in range(10):
    acc = 100*n_class_correct[i]/n_class_samples[i]
    print(f'accuracy of {classes[i]}: {acc} %')

epoch 1/4, step 2000/12500, loss = 2.2201
epoch 1/4, step 4000/12500, loss = 2.4408
epoch 1/4, step 6000/12500, loss = 2.1348
epoch 1/4, step 8000/12500, loss = 2.1138
epoch 1/4, step 10000/12500, loss = 1.8268
epoch 1/4, step 12000/12500, loss = 1.9978
epoch 2/4, step 2000/12500, loss = 2.0607
epoch 2/4, step 4000/12500, loss = 1.5301
epoch 2/4, step 6000/12500, loss = 1.6085
epoch 2/4, step 8000/12500, loss = 2.7176
epoch 2/4, step 10000/12500, loss = 1.9907
epoch 2/4, step 12000/12500, loss = 1.5659
epoch 3/4, step 2000/12500, loss = 0.9518
epoch 3/4, step 4000/12500, loss = 1.8956
epoch 3/4, step 6000/12500, loss = 1.4369
epoch 3/4, step 8000/12500, loss = 2.5217
epoch 3/4, step 10000/12500, loss = 1.3141
epoch 3/4, step 12000/12500, loss = 1.4377
epoch 4/4, step 2000/12500, loss = 1.1229
epoch 4/4, step 4000/12500, loss = 1.1527
epoch 4/4, step 6000/12500, loss = 1.0860
epoch 4/4, step 8000/12500, loss = 1.9122
epoch 4/4, step 10000/12500, loss = 0.8188
epoch 4/4, step 12000/12500